# Day 8: Handheld Halting
https://adventofcode.com/2020/day/8

In [1]:
inputLines = lines <$> readFile "input/day08.txt"

In [2]:
testInput = [ "nop +0"
            , "acc +1"
            , "jmp +4"
            , "acc +3"
            , "jmp -3"
            , "acc -99"
            , "acc +1"
            , "jmp -4"
            , "acc +6"]

# Part 1

Define a data type for instructions.

In [3]:
data Instruction = Nop Int | Jmp Int | Acc Int deriving (Show)

Specify how the instrutions change `index` (the index of the next instruction in the array that holds all instructions) and the accumulator.

In [4]:
execute :: Instruction -> (Int, Int) -> (Int, Int)
execute (Nop _) (index, acc) = (succ index, acc)
execute (Jmp delta) (index, acc) = (index + delta, acc)
execute (Acc delta) (index, acc) = (succ index, acc + delta)

Parse a single instruction.

In [5]:
import Text.Regex.PCRE  -- install with 'stack install regex-pcre'

parseInstruction :: String -> Instruction
parseInstruction line =
    let
    
        [_, instruction, sign, number] = head (line =~ "(acc|jmp|nop) ([+-])([0-9]+)" :: [[String]])
        
        parsedNumber = 
            (read number :: Int) *
            (case sign of "+" -> 1
                          "-" -> -1)
        
        parsedInstruction =
            case instruction of "acc" -> Acc
                                "jmp" -> Jmp
                                "nop" -> Nop
    in
        parsedInstruction parsedNumber

A program is represented by an array of instructions.

In [6]:
import Data.Array
type Program = Array Int Instruction

Parse an entire program.

In [7]:
parseProgram :: [String] -> Program
parseProgram programLines = listArray (0, pred . length $ programLines)
                          $ map parseInstruction programLines

Run a program until either
* An infinite loop is detected. The result is then `Left acc`, where `acc` is the accumulator value just before an instruction would be executed for the second time.
* The program ends regularly because the next instruction would be immediately after the last instruction in the program. In this case, the result is `Right acc`, where `acc` is the final accumulator value.

In [8]:
import qualified Data.Set as Set

runProgram :: Program -> Either Int Int
runProgram program =
    let
        terminationIndex = succ . snd . bounds $ program

        run :: Set.Set Int -> (Int, Int) -> Either Int Int
        run alreadyExecuted (index, acc)
            | index == terminationIndex = Right acc
            | Set.member index alreadyExecuted = Left acc
            | otherwise = run (Set.insert index alreadyExecuted)
                              (execute (program ! index) (index, acc))

    in
        run Set.empty (0, 0)

In [9]:
solution1 = runProgram . parseProgram

Verify given example.

In [10]:
solution1 testInput

Left 5

## Solution, part 1

In [11]:
solution1 <$> inputLines

Left 1928

# Part 2

Find out which instructions are possibly corrupted, i.e., at which indices there are `jmp` or `nop` instructions.

In [12]:
isJmpOrNop :: Instruction -> Bool
isJmpOrNop (Jmp _) = True
isJmpOrNop (Nop _) = True
isJmpOrNop _ = False

In [13]:
possiblyCorruptedIndices :: Program -> [Int]
possiblyCorruptedIndices = map fst . filter (isJmpOrNop . snd) . assocs

Modify an instruction by replacing `jmp` by `nop` and vice versa.

In [14]:
changeInstruction :: Instruction -> Instruction
changeInstruction (Jmp n) = Nop n
changeInstruction (Nop n) = Jmp n
changeInstruction other = other

Create all possible modifications of a program by
* finding out which instructions are possibly corrupted, and
* create a modified program for each of these where the respective instruction is changed.

In [15]:
import Control.Lens (over, element) -- install with 'stack install lens'

modifiedPrograms :: Program -> [Program]
modifiedPrograms program =
    map (\index -> over (element index) changeInstruction program) indices
    
    where
        indices = possiblyCorruptedIndices program

In [16]:
import Data.Either (rights)

solution2 = head . rights . map runProgram . modifiedPrograms . parseProgram

Verify given example.

In [17]:
solution2 testInput

8

## Solution, part 2

In [18]:
solution2 <$> inputLines

1319